In [2]:
import pandas as pd, numpy as np, re, nltk, string
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from sklearn.preprocessing import OrdinalEncoder
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import BernoulliNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Part 1: Initializing the machine learning categorization algorithm**

**Set constants**

In [48]:
DATA_FILE = 'sample df.xlsx'
UNCATEGORIZED_DATA = 'uncategorized.xlsx'
EXPORT_FILE = 'predicted categories.csv'
MAX_FEATURES = 1000
CUTOFF_POINT = .5
SAMPLE_SIZE = 1
ALPHA_VALUE = .1

**Import sample dataset** 

In [31]:
df = pd.read_excel(DATA_FILE)
sampled = df.groupby("Category ID").sample(frac=SAMPLE_SIZE)
sampled.iloc[[1]]

,Unnamed: 0,Product ID,Product Title,Brand,Supplier,Supplier Full Category Paths,Ofbiz Full Category Paths,Category ID,Category Name,Marketing Description,Technical Description,Unnamed: 11
1,2341,10772044,Savitch Malley Word Processor Subscription Ser...,Savitch,Lahey,Word Processing>One Year Subscriptions,Word Processing>Subscriptions>One Year Subscri...,36590,One Year Word Processing Subscriptions,Meet all your word processor needs with new cu...,<ul><li>Product Type: Word Processing</li><li>...,NaN


**Filter dataframe to only contain relevant columns**

In [32]:
df = sampled.filter(['Category ID', 'Marketing Description', 'Technical Description', 'Brand', 'Supplier', 'Product ID'])
df.iloc[[1]]

,Category ID,Marketing Description,Technical Description,Brand,Supplier,Product ID
1,36590,Meet all your word processor needs with new cu...,<ul><li>Product Type: Word Processing</li><li>...,Savitch,Lahey,10772044


**Combine marketing and technical descriptions into Marketing Technical Description column**

In [33]:
df['Marketing Technical Description'] = df['Marketing Description'].astype(str) + " " + df['Technical Description'].astype(str)
df.iloc[[1]]

,Category ID,Marketing Description,Technical Description,Brand,Supplier,Product ID,Marketing Technical Description
1,36590,Meet all your word processor needs with new cu...,<ul><li>Product Type: Word Processing</li><li>...,Savitch,Lahey,10772044,Meet all your word processor needs with new cu...


**Iterate through dataframe and clean combined Marketing Technical Description column**

**(Remove HTML tags, remove punctuation, make lowercase, stem words, and tokenize words)**

In [11]:
def iterate_clean(df):
  stemmer = PorterStemmer()
  stop_words = set(stopwords.words('english'))
  tags = re.compile('<.*?>')
  num = re.compile('\\d+')
  clean_desc = []
  index = 0

  for i in range(len(df)):
    try:
      desc = df.loc[index, 'Marketing Technical Description']

    except KeyError:
      desc = " "

    finally:
        no_name = re.sub("Name: Marketing Technical Description, dtype: object", '', str(desc))
        no_tags = re.sub(tags, ' ', no_name)
        no_num = re.sub(num, ' ', no_tags)
        no_punc = no_num.translate(str.maketrans('', '', string.punctuation))
        wordlist = no_punc.split()
        lowercase_words = []
        
        for i in wordlist:
          if not i.lower() in stop_words:
            if len(i)>1:
              lowercase_words.append(i.lower())
        
        stemmed = {}
        index2 = 0
        
        for i in lowercase_words:
          stemmed[i] = stemmer.stem(lowercase_words[index2])
          joined = " ".join(stemmed)
          index2 += 1
            
        clean_desc.append(joined)
        index += 1

    clean_series = pd.Series(clean_desc)
    df['Cleaned Marketing Technical Description'] = clean_series

In [34]:
iterate_clean(df)

In [35]:
df['Marketing Technical Description'].loc[1]

'Meet all your word processor needs with new custom software for top-security needs. Simply install, open, and start typing! <ul><li>Product Type: Word Processing</li><li>License Type: 1 year</li><li>Compatible With: Windows, Mac, Linux</li></ul>'

In [36]:
df['Cleaned Marketing Technical Description'].loc[1]

'meet word processor needs new custom software topsecurity simply install open start typing product type processing license year compatible windows mac linux'

**Create test/train split**

In [37]:
X = df.loc[:, ~df.columns.isin (['Category ID', 'Marketing Description', 'Marketing Technical Description', 'Technical Description'])]

y = df['Category ID']

X_to_train, X_to_test, y_train, y_test = train_test_split(X, y)

**Transform cleaned description into a matrix of token counts for each row of data**

In [60]:
matrix = CountVectorizer(max_features=MAX_FEATURES, decode_error='ignore', strip_accents='ascii')

vectors1 = matrix.fit_transform(X_to_train['Cleaned Marketing Technical Description'].values.astype(str)).toarray()
vectordf1 = pd.DataFrame(vectors1)
vectordf1.columns = vectordf1.columns.astype(str)

vectors2 = matrix.transform(X_to_test['Cleaned Marketing Technical Description'].values.astype(str)).toarray()
vectordf2 = pd.DataFrame(vectors2)
vectordf2.columns = vectordf2.columns.astype(str)

vectordf1.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Encode Brand, Supplier, & Product ID as ordinal dummy variables.**

In [61]:
enc = OrdinalEncoder()
to_enc1 = X_to_train.filter(['Brand', 'Supplier', 'Product ID'])
encoded1 = enc.fit_transform(to_enc1)
enc_df_1 = pd.DataFrame(encoded1, columns=['Brand', 'Supplier', 'Product ID'])
merged1 = vectordf1.join(enc_df_1)
X_train = merged1.fillna(0)

to_enc2 = X_to_test.filter(['Brand', 'Supplier', 'Product ID'])
encoded2 = enc.fit_transform(to_enc2)
enc_df_2 = pd.DataFrame(encoded2, columns=['Brand', 'Supplier', 'Product ID'])
merged2 = vectordf2.join(enc_df_2)
X_test = merged2.fillna(0)

X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,993,994,995,996,997,998,999,Brand,Supplier,Product ID
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,67.0,41.0,1181.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,100.0,41.0,7273.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,60.0,38.0,763.0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,104.0,23.0,1709.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,100.0,41.0,7347.0


**Naive Bayes**

In [62]:
classifier = BernoulliNB(alpha=ALPHA_VALUE)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy = round(accuracy*100,1)
accuracy

92.2

**Confusion matrix**

In [63]:
confusion = confusion_matrix(y_test, y_pred)
confusion_df = pd.DataFrame(confusion)
confusion_df

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,0,0,0
1,0,293,0,0,13,0,13,7,0,0,0
2,0,0,126,0,0,55,0,0,0,0,0
3,0,0,0,115,0,0,1,0,0,0,0
4,0,0,0,0,916,0,8,20,0,0,0
5,0,0,7,0,19,190,3,2,0,0,0
6,7,0,0,0,20,0,330,24,1,0,2
7,0,0,0,0,0,0,0,20,0,0,0
8,4,0,0,0,0,0,0,14,520,0,2
9,0,0,0,0,0,0,0,1,0,135,0


**Part 2: Apply algorithm to uncategorized data**

**Import & preprocess uncategorized data**

In [64]:
uncat_df = pd.read_excel(UNCATEGORIZED_DATA)
uncat_df.head(1)

,Unnamed: 0,Product ID,Product Title,Brand,Supplier,Supplier Full Category Paths,Ofbiz Full Category Paths,Category ID,Category Name,Marketing Description,Technical Description,Unnamed: 11
0,6543,10763000,Browne/King - GG100-253C - Network Cable,Browne/King,Wilcom,Cables>Network Cables>Ethernet Cable,Cables>Network Cables>Ethernet Cable,14132.0,Ethernet Cables,The Browne/King ethernet cable exceeds expecta...,NaN,Cable length: 5 ft Cable diameter: 0.5 in Data...


In [65]:
uncat_df = uncat_df.filter(['Category ID', 'Marketing Description', 'Technical Description', 'Brand', 'Supplier', 'Product ID'])
uncat_df['Marketing Technical Description'] = uncat_df['Marketing Description'].astype(str) + " " + uncat_df['Technical Description'].astype(str)

uncat_encode = uncat_df.filter(['Brand', 'Supplier', 'Product ID'])
uncat_encode = enc.fit_transform(uncat_encode)
uncat_encode = pd.DataFrame(uncat_encode, columns=['Brand', 'Supplier', 'Product ID'])

iterate_clean(uncat_df)

vector_uncat = matrix.transform(uncat_df['Cleaned Marketing Technical Description'].values.astype(str)).toarray()
vector_uncat = pd.DataFrame(vector_uncat)

dropped_uncat = uncat_df.drop(['Brand', 'Supplier', 'Product ID'], axis=1)
merged_uncat = dropped_uncat.join(vector_uncat)
merged_uncat = merged_uncat.join(uncat_encode)
merged_uncat = merged_uncat.fillna(0)

valid_X = merged_uncat.loc[:, ~merged_uncat.columns.isin (['Category ID', 'Cleaned Marketing Technical Description', 'Marketing Description', 'Marketing Technical Description', 'Technical Description'])]
valid_X.head()

,0,1,2,3,4,5,6,7,8,9,...,993,994,995,996,997,998,999,Brand,Supplier,Product ID
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,9.0,15.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.0,6.0,14.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.0,7.0,13.0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,6.0,3.0,12.0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,3.0,8.0


**Predicted y values**

In [66]:
valid_y = classifier.predict(valid_X).astype(int)
valid_y

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliNB was fitted with feature names
  "X does not have valid feature names, but"


array([230716,  50008,  50008,  50008,  50013,  50013, 226590,  50023,
        50023,  50023,  60184, 234751,  60184, 226590, 226590,  60184,
        60184, 230716, 230716, 234751, 234751, 234751, 240690, 240690,
       240690, 240707, 240707, 240707])

**De-encode ordinal variables for predicted results**

In [67]:
y_df = pd.DataFrame(valid_y, dtype='int64').astype(int)
y_df.columns = ['Predicted Category ID']

predict_df = valid_X[['Brand', 'Supplier', 'Product ID']]
unencoded = enc.inverse_transform(predict_df)
unencoded = pd.DataFrame(unencoded, columns=['Brand', 'Supplier', 'Product ID'])
unencoded.iloc[[0]]

,Brand,Supplier,Product ID
0,Browne/King,Wilcom,10763000


**Add columns with predicted probability of each category**

In [68]:
valid_prob = classifier.predict_proba(valid_X)
cols = classifier.classes_.astype(int)
valid_prob_df = pd.DataFrame(valid_prob, columns=cols).add_prefix('Probability Of Category ')
valid_prob_df = round(valid_prob_df*100, 5)
valid_prob_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliNB was fitted with feature names
  "X does not have valid feature names, but"


,Probability Of Category 14132,Probability Of Category 36590,Probability Of Category 50008,Probability Of Category 50013,Probability Of Category 50023,Probability Of Category 60184,Probability Of Category 226590,Probability Of Category 230716,Probability Of Category 234751,Probability Of Category 240690,Probability Of Category 240707,Probability Of Category 242004
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.99903,0.00097,0.0,0.0,0.0
1,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0
2,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0
3,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0
4,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0


**If probability of any category meets a cutoff point, label it as "categorized"**

In [69]:
cat_or_not = []

for i in valid_prob:
  not_cat = True
  while not_cat:
    for j in i:
      if j > CUTOFF_POINT:
        cat_or_not.append('categorized')
        not_cat = False
else:
  cat_or_not.append('uncategorized')
      
cat_df = pd.Series(cat_or_not).rename('Categorized or Not')
cat_df.head()

0    categorized
1    categorized
2    categorized
3    categorized
4    categorized
Name: Categorized or Not, dtype: object

**Join validation data to predicted results**

In [70]:
final_pred = pd.concat([unencoded, y_df, cat_df, valid_prob_df], axis=1)
final_pred.iloc[[0]]

,Brand,Supplier,Product ID,Predicted Category ID,Categorized or Not,Probability Of Category 14132,Probability Of Category 36590,Probability Of Category 50008,Probability Of Category 50013,Probability Of Category 50023,Probability Of Category 60184,Probability Of Category 226590,Probability Of Category 230716,Probability Of Category 234751,Probability Of Category 240690,Probability Of Category 240707,Probability Of Category 242004
0,Browne/King,Wilcom,10763000,230716.0,categorized,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.99903,0.00097,0.0,0.0,0.0


**Export results to CSV**

In [71]:
final_pred.to_csv(EXPORT_FILE)